<a href="https://colab.research.google.com/github/AlexanderTS1/BIGDATA2021/blob/main/ACTIVIDAD_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Actividad 03 Reglas de Asociación
```
Docente     : Carlos Fernando Montoya Cubas
Estudiante  : Alexander Ttito Saya
Lugar       : Cusco, Perú, 2021.

```
---

## Librerias a utilizar 🟠


In [ ]:
#librerias

import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import collections

## Implementacion de modulos para generar tuplas de k elementos del dataset y módulos para implementacion de apriori 🟠

In [ ]:
#convertir lista de musicas en arreglo de areglos
playlists=[datos[i] for i in range (0,len(datos))]

In [ ]:
#mostramos tamaño original de datos
print("Longitud  de spotify.npy: ",len(playlists))

Longitud  de spotify.npy:  10000


In [ ]:
# Mostramos los areglos generados
print(playlists[0:100])
# Mostramos el tamaño de este del subarreglo
print("El tamaño del arreglo es:",len(playlists[0:100][50]))

[['Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Right Where You Want Me - Radio Edit Version', 'Beautiful Soul', "Leavin'", 'Me & U', 'Ice Box', 'Sk8er Boi', 'Run It!', 'Check On It - feat. Bun B and Slim Thug', "Jumpin', Jumpin'", 'Soak Up The Sun', 'Where Is The Love?', "Stacy's Mom", 'Just The Girl', 'Yo (Excuse Me Miss)', 'Year 3000', 'Lip Gloss', 'Everytime We Touch - Radio Edit', 'Whatcha Say', 'Miss Independent', 'Party In The U.S.A.', 'The Great Escape', 'Replay', 'Forever', 'Your Love Is My Drug', 'Closer', 'One Less Lonely Girl', 'Paper Planes', 'Mr. Brightside', 'All The Small Things', 'Beep', 'Somebody To Love', 'Dirty Little Secret', 'Baby', 'A Thousand Miles', 'Livin on Sunday', 'See You Again', 'How Do You Sleep? - Featuring Ludacris', 'This Is Me', 'My Happy Ending', 'Check Yes Juliet', 'The Great Escape'], ['Eye of t

In [ ]:
#Esta funcion permitira eliminar aquellas reglas que se van a repetir 
def EliminarrReglasRepetidas(X,Y):
    #Se elimina la combinacion igual 
    validos = [True]*len(X)
    for k in range(len(X)-1):
        '''Si el valor en la posicion k es valido se verifica si ya existe'''
        if(validos[k]):
            '''Aqui se comprueba si la regla es valida  recoorirndo todo'''
            for j in range(k+1,len(X)):
                'si la regla es igual a X e Y  es no Valido'
                if(validos[j] and 
                    X[k] == X[j] and
                    Y[k] == Y[j]):
                    validos[j]=False
    #Eliminamos los no validos, tanto para X como Y
    X = list(it.compress(X,validos))
    Y = list(it.compress(Y,validos))
    return X,Y

In [ ]:
#Funcion para obtener la interseccion de conjunto de items
def get_intersection_items(subset,canciones_playlist):
    group_songs = []
    for song in subset:
        group_songs.append(canciones_playlist[song])
    return len(set.intersection(*group_songs))

In [ ]:
def gentuplas(dataset,r):
  #Con la funcion combinations() generamos tuplas de longitud r  y retornamos con funccion set()  para evitar elemento repetidos
   return set(itertools.combinations(dataset,r))

## Definir umbrales 🟠

In [ ]:
#definimos umbrales en base a ejemplo realizado en practicas del curso

veces_minimas_aparece_en_lista=100
soporte_min = veces_minimas_aparece_en_lista / len(playlists)
confianza_min =0.5
sustento_min =1.5

##1.  Implementar el algoritmo Apriori 🟠⏮ 

### Funcion Modulo get_frequent_itemsets ⏭

---



```
- ENTRADA: esctructura de datos que contiene a las playlists y un umbral de soporte.
- SALIDA: una estructura con los itemsets frecuentes, bajo un umbral mínimo de confianza.
```


In [ ]:
def get_frequent_itemsets(playlists, min_support):
  #Convertimos dataset en conjunto de listas  
  playlists = list(playlists.item().values())
  playlists = [set(playlist) for playlist in playlists]
  canciones = [item for sublist in playlists for item in sublist]
  #Se obtiene su cardinalidad o frecuencia de cada cancion
  count_song = pd.Series(data=canciones).value_counts().to_dict()
  #Geramos un diccionario de canciones y su respectivos indices
  songs = collections.defaultdict(set)
  for index, playlist in enumerate(playlists):
      for cancion in playlist:
          songs[cancion].add(index)
  #Obtenemos los itemsets que pasan la condicion de umbral min
  get_n_items = {cancion: veces for cancion, veces in count_song.items() if veces / len(playlists) >= min_support}
  itemset = [{cancion} for cancion in get_n_items.keys()]
  #Se obtienen los itemsets mas frecuentes
  get_frequent_item = {}
  get_frequent_item[1] = sorted(get_n_items.items(), key=lambda x: x[1], reverse=True)
  count_frequent_itemsets = []
  n = 2
  #Se alamacena el valor de "itemset" en la variable actual
  actual = itemset   
  while len(actual) != 0:
    #Generar itemsets "candidatos" y eliminar los itemsets que no son frecuentes y los almacenamos en un diccionario
    diccionario_combinaciones = set() 
    m = n - 2
    for candidato in actual: 
      candidato = list(candidato)
      for aux_candidato in actual:
        aux_candidato = list(aux_candidato)
        #inciializamos una bandera 
        bandera = True
        for i in range(n - 2):
          if candidato[i] != aux_candidato[i]:
              bandera = False
              break
        if not bandera:
          continue
        if candidato[n - 2] < aux_candidato[n - 2]:
          c = candidato + [aux_candidato[n - 2]]
          c = frozenset(sorted(c))
          diccionario_combinaciones.add(c)
    #Generar el itemset que cumplan con la condicion del umbral minimo de soporte
    contador_combinaciones = {}   
    playlist_tamanio = len(playlists)   
    for candidato in diccionario_combinaciones:
        playlists_inter = []
        for song in candidato:
          playlists_inter.append(songs[song])
        contador_combinaciones[candidato] = len(set.intersection(*playlists_inter))
    contador_l = {subset: veces for subset, veces in contador_combinaciones.items() if veces / playlist_tamanio >= min_support} 
    diccionario = contador_l.keys() 
    count_frequent_itemsets.extend(diccionario)
    #se ordena la lista con la funcion Sort()
    get_frequent_item[n] = sorted(contador_l.items(), key=lambda x: x[1], reverse=True)
    #se actulaaliza el parametro y el valor de la variable
    n += 1
    actual = diccionario
  #Con libreira pandas organizar los itemset  como un dataframe
  itemsetsdataframe = pd.DataFrame([item for sublist in get_frequent_item.values() for item in sublist]).round(3)
  itemsetsdataframe.columns = ["itemsets", "count_support"]
  itemsetsdataframe["support"] = itemsetsdataframe["count_support"] / len(playlists)
  #Se genera una array para retornar un  con 4 datos
  supergrupo = []
  #En la primera posicion estaran los itemsets mas frecuentes seguido por los playlist, canciones y los itemsets frecuents
  supergrupo.append(itemsetsdataframe)
  supergrupo.append(playlists)
  supergrupo.append(songs)
  supergrupo.append(count_frequent_itemsets)
  #Se retorna todo los valores acumulados en supergrupo de 4 valores
  return supergrupo

### Funcion Modulo generate_association_rules ⏭

---



```
- ENTRADA: los itemsets frecuentes generados por la función anterior 


- RETORNA: Las reglas de asociación. Se le puede entregar umbrales de confianza o lift para las
reglas que se retornarán. 
Por ejemplo, si se llama esta función con los ar- gumentos 
confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que
cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2
```


In [ ]:

def generate_association_rules(frequent_itemsets, confidence, lift):
  reglas = []
  for itemset in frequent_itemsets[3]:
    #Se forman todas las combinaciones posibles segun la logica de asociacion
    count_itemset = get_intersection_items(itemset,frequent_itemsets[2])
    itemset_support = count_itemset / len(frequent_itemsets[0])
    for i in range(1, len(itemset) + 1):
        #utilizando la funcion Combination() de itertools 
        for x in it.combinations(itemset, i):
            x = set(x)
            y = set(itemset) - x
            x_support = get_intersection_items(x,frequent_itemsets[2]) / len(frequent_itemsets[0])
            x_y_support = get_intersection_items(x.union(y),frequent_itemsets[2]) / len(frequent_itemsets[0])
            rule_confidence = x_y_support / x_support
            if len(x) > 0 and len(y) > 0:
                y_support = get_intersection_items(y,frequent_itemsets[2]) / len(frequent_itemsets[0])
                rule_lift = x_y_support / (x_support * y_support)
                reglas.append((x, y, rule_confidence, x_y_support, rule_lift))

  #Utilziando la libreria pandas generar dataframe para organizar los resultados
  association_rules = pd.DataFrame(data=reglas, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  association_rules["X"] = list(map(tuple, association_rules["X"]))
  association_rules["Y"] = list(map(tuple, association_rules["Y"]))
  #Ordenar (con los valores de Sort de un dataframe) las reglas de asociacion por confianza y mostrar los 10 mejores
  n=10 #se pide 10 mejores reglas de asociacion
  order_by="confidence"
  confidence_df = association_rules.sort_values(order_by, ascending=False).head(n)
  #Ordenar(con los valores de Sort de un dataframe) las reglas de asociacion para lift y mostrar los 10 mejores
  order_by="lift"
  lift_df2 = association_rules.sort_values(order_by, ascending=False).head(n)
  #Hallar las reglas de asociacion que pasen el umbral
  association_rules["len_itemset"] = association_rules.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  association_rules=association_rules.sort_values(by="len_itemset", ascending=False)
  both_rules = association_rules[(association_rules["confidence"] >= confidence) & (association_rules["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules

## 2. Aplicar el algoritmo y obtener reglas de asociación 🟠⏮ 

### Cargar data set  spotify.npy 🟠

In [ ]:
#utilizando la libreria numpy leemos data set spotyfy.npy
datos = np.load("spotify.npy", allow_pickle = True)


### Obtener los itemset frecuentes 🟠

In [ ]:
#Aqui se obtiene los data set fercuente
itemsets = get_frequent_itemsets(datos,0.01)
itemsets[0].sort_values(by="support").head()

,itemsets,count_support,support
1023,"(Broccoli (feat. Lil Yachty), Bad and Boujee (...",100,0.01
999,"(iSpy (feat. Lil Yachty), Swang)",100,0.01
998,"(Drowning (feat. Kodak Black), goosebumps)",100,0.01
997,"(Jumpman, One Dance)",100,0.01
996,"(pick up the phone, Bad and Boujee (feat. Lil ...",100,0.01


### Obtner las reglas de asociacion en base a los itemset frecuentes obtenidos 🟠

In [ ]:
#Aqui se obtiene las reglas de asocioacion en base a los itenset calculados en la seccion anterior
#Se tiene los siguientes valores: confidence = 0.5 y lift = 1.5
reglas_confidence,reglas_lift,reglas_ambos = generate_association_rules(itemsets,0.5,1.5)
reglas_confidence

,X,Y,confidence,support,lift
391,"(Mask Off, DNA.)","(HUMBLE.,)",0.909,0.098,2.002
439,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904
270,"(DNA.,)","(HUMBLE.,)",0.823,0.186,1.811
397,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.128,1.770
446,"(Broccoli (feat. Lil Yachty), Bounce Back)","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301
379,"(Slippery (feat. Gucci Mane), XO TOUR Llif3)","(HUMBLE.,)",0.765,0.099,1.685
427,"(Tunnel Vision, XO TOUR Llif3)","(HUMBLE.,)",0.750,0.100,1.652
410,"(Mask Off, Congratulations)","(HUMBLE.,)",0.747,0.118,1.645
373,"(Mask Off, goosebumps)","(HUMBLE.,)",0.743,0.107,1.637
385,"(Mask Off, Bounce Back)","(HUMBLE.,)",0.743,0.099,1.635


## 3. Explicar las reglas obtenidas 🧠

```
Deberás seleccionar 4 reglas y comentar su calidad de acuerdo a los diferentes indicadores disponibles (support, confidence y lift). Además, según el género y/o el artista
de las canciones (que puedes buscar según el nombre de la canción) deberás darle una
breve interpretación a las reglas.
```

In [ ]:
#Obtener las 4 reglas, en este casso se elige las 4 mejores reglas segun variable "confidence"
reglas=reglas_confidence.head(4)
reglas


,X,Y,confidence,support,lift
391,"(Mask Off, DNA.)","(HUMBLE.,)",0.909,0.098,2.002
439,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904
270,"(DNA.,)","(HUMBLE.,)",0.823,0.186,1.811
397,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.128,1.770


✅ En este caso estoy filtrando las 4 mejores reglas segun mi variable "confidence"  y obtenemos las mejores k-tuplas.

✅ Primero calculando las reglas de asociación con los valores de confidence = 0.5 y lift = 1.5  se obtiene resultados satifactorios ya que todos valores calulados de confidenc y litf son mayores.
 
Ahora se explica cada una de las reglas:

🔴**REGLA 1**:    

(Mask Off, DNA.)	->(HUMBLE.,)	, Quiere decir que si se escucha  "Mask Off,DNA" puede que escuche  "Humble" como siguiente cancion, en mi opinion esto es debido  a que pertenecen al mismo genero Hip-Hop/Rap. El valor de confianza alcanza a  0.909 superando con mucho al valor predefinido del umbral.

🔴**REGLA 2:**  

(XO TOUR Llif3, DNA.) ->	(HUMBLE.,) Quiere  decir que escuchar 	XO TOUR Llif3(genero Emo rap) , DNA pue escuchar tambien HUMBLE(genero hip-hop/rap).  Haciendo referencia al anterior comentario estas canciones estan dentro de un genero de musicas es por eso la regla de asociacion. La confianza es de 0.864 y supera al valor de umbral predefinido.

🔴**REGLA 3:** 

 (DNA.) ->	(HUMBLE.,) Quiere  decir que escuchar 	DNA(genero Hip-Hop/rap) , DNA pue escuchar tambien HUMBLE(genero hip-hop/rap).  Haciendo referencia al anterior comentario estas canciones estan dentro de un genero de musicas es por eso la regla de asociacion. La confianza es de 0.823 y supera al valor de umbral predefinido.

🔴**REGLA 4:** 

 (XO TOUR Llif3, DNA.) ->	(HUMBLE.,)  como ya se ha mencionado en los comentarios de la reglas anteriores estas canciones estan dentro de un mismo genero de musica La confianza es de 0.804 y supera al valor de umbral predefinido.

✅✅ **APRECIACION:** EN ESTAS CUATRO REGLAS SE REPITEN CASI MAS MISMA CANCIONES ESTO ES DEBIDO A QUE PERTENCEN AL MISMO GENERO, ADEMAS EN LOS AÑOS 2016 Y 2017 TUVIERON MAYOR ACEPTACION POR EL PUBLICO TAMBIEN ES IMPORATANTE MENCIONAR QUE LOS ARTITAS DE TODAS LAS CANCIONES OBTENIDAS EN LAS 4 REGLAS SON HOMBRES.


▶  En este caso estoy filtrando las 4 mejores reglas segun mi variable "confidence"  y obtenemos las mejores k-tuplas. Es importante que en estas prubas se utiliza el valor de lif

**REGLA 1:** (Mask Off, DNA.)	->(HUMBLE.,)	, quiere decir que si se escuha  "Mask Off,DNA" implica escuchar "Humble" como siguiente cancion, en mi opinion esto es debido  a que pertenecen al mismo genero. El calor de confianza alcanza a  0.909 superando con mucho al valor predefinido del umbrl

**REGLA 2:**  (XO TOUR Llif3, DNA.) ->	(HUMBLE.,) es decir que escuchar 	XO TOUR Llif3, DNA implica escuchar tambien HUMBLE.  haciendo referencia al anterior comentario estas canciones pertencen al mismo genero trap(y artistas que componen canciones de este genero) y   lo mismo pasa en las posteriores reglas(3 y 4) superendo  con mucho al umbral predefinido que tiene el valor de 0.5